In [1]:
!pip install git+https://github.com/Priesemann-Group/covid19_inference_forecast.git

  Cloning https://github.com/Priesemann-Group/covid19_inference_forecast.git to /private/var/folders/qr/hdfcfrpx0hj50l5g7lwg7ygc0000gn/T/pip-req-build-f13d7n0s
  Running command git clone -q https://github.com/Priesemann-Group/covid19_inference_forecast.git /private/var/folders/qr/hdfcfrpx0hj50l5g7lwg7ygc0000gn/T/pip-req-build-f13d7n0s
  on a case-insensitive filesystem) and only one from the same
  colliding group is in the working tree:

    'DISCLAIMER.md'
    'disclaimer.md'
  Created wheel for covid19-inference: filename=covid19_inference-0.0.10-py3-none-any.whl size=27625 sha256=1192e0641840717ac8fa8daac252628b058301fe2baddf7533141747c88e3f20
  Stored in directory: /private/var/folders/qr/hdfcfrpx0hj50l5g7lwg7ygc0000gn/T/pip-ephem-wheel-cache-2mt6jha4/wheels/cc/33/51/375e62e25122e2286fa440949767c3f7f3e2e75f7a2cd4ca7e
Successfully built covid19-inference


In [2]:
import sys
sys.path.append('../..')
import datetime
import pickle

import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib

try: 
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append('..')
    import covid19_inference as cov19
    


confirmed_cases = cov19.get_jhu_confirmed_cases()

date_data_begin = datetime.datetime(2020,3,8)
date_data_end   = cov19.get_last_date(confirmed_cases)
num_days_data = (date_data_end-date_data_begin).days
diff_data_sim = 12 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_future = 365
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_data_end   + datetime.timedelta(days = num_days_future)
num_days_sim = (date_end_sim-date_begin_sim).days

country = 'US'

cases_obs = cov19.filter_one_country(confirmed_cases, country,
                                     date_data_begin, date_data_end)

prior_date_mild_dist_begin =  datetime.datetime(2020,3,23)
prior_date_strong_dist_begin =  datetime.datetime(2020,5,11)

change_points = [dict(pr_mean_date_begin_transient = prior_date_mild_dist_begin,
                      pr_median_lambda = 0.3,
                      pr_sigma_lambda = 1),
                 dict(pr_mean_date_begin_transient = prior_date_strong_dist_begin,
                      pr_sigma_date_begin_transient = 1,
                      pr_median_lambda = 0.35,
                      pr_sigma_lambda = 1),
                ]

traces = []
models = []

model = cov19.SEIR_with_extensions(np.diff(cases_obs),
                                    change_points,
                                    date_begin_sim,
                                    num_days_sim,
                                    diff_data_sim, 
                                    N=328e6,
                                    priors_dict=None,
                                    with_random_walk=False,
                                    weekends_modulated=True,
                                    weekend_modulation_type = 'abs_sine')

models.append(model)

traces.append(pm.sample(model=model, init='advi', draws=1000))

INFO: This code is not longer developed, use our new module instead: github.com/Priesemann-Group/covid19_inference
pr_beta_I_begin was set to default value 100
pr_beta_E_begin_scale was set to default value 10
pr_median_lambda_0 was set to default value 2
pr_sigma_lambda_0 was set to default value 0.7
pr_median_mu was set to default value 0.3333333333333333
pr_sigma_mu was set to default value 0.3
pr_median_delay was set to default value 5
pr_sigma_delay was set to default value 0.2
scale_delay was set to default value 0.3
pr_beta_sigma_obs was set to default value 10
pr_mean_median_incubation was set to default value 5
pr_sigma_median_incubation was set to default value 1
sigma_incubation was set to default value 0.418
week_end_days was set to default value (6, 7)
pr_mean_weekend_factor was set to default value 0.7
pr_sigma_weekend_factor was set to default value 0.17
pr_sigma_date_begin_transient of change point 0 was set to default value 3
pr_median_transient_len of change point 0 w

Auto-assigning NUTS sampler...
Initializing NUTS using advi...


Convergence achieved at 29500
Interrupted at 29,499 [14%]: Average Loss = 1,607.2
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [offset_modulation_rad, weekend_factor, median_incubation, sigma_obs, delay, mu, transient_len_1, transient_len_0, transient_begin_1, transient_begin_0, lambda_2, lambda_1, lambda_0, E_begin, E_begin_scale, I_begin]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 147332 seconds.
There were 24 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


In [3]:
print(pm.loo(traces[0], model))

/Users/jessica/opt/anaconda3/lib/python3.7/site-packages/arviz/data/io_pymc3.py:89: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


Computed from 2000 by 117 log-likelihood matrix

         Estimate       SE
elpd_loo -1048.90    10.32
p_loo       12.63        -
------

Pareto k diagnostic values:
                         Count   Pct.
(-Inf, 0.5]   (good)      117  100.0%
 (0.5, 0.7]   (ok)          0    0.0%
   (0.7, 1]   (bad)         0    0.0%
   (1, Inf)   (very bad)    0    0.0%


The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if
you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive
accuracy.


In [14]:
import figures_revised
import figures

In [38]:
a=np.median(traces[0]['lambda_0'])
b=np.median(traces[0]['lambda_1'])
c=np.median(traces[0]['lambda_2'])

In [39]:
print(a,b,c)

5.121073289232985 0.45612500273221146 0.6326264906154196


In [47]:
a=np.cumsum(np.insert(traces[0]["new_cases"][:, num_days_data : num_days_data + 100
    ], 0, 0, axis=1), axis=1) + cases_obs[0]

In [48]:
np.median(a[:,-1]) 

46282274.47851786